In [6]:
%load_ext autoreload
%autoreload 2

import os,sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import util as yu
from util import *
import util_NpiScatteringWilson as yun

yu.setpath('analysis_c2pt_pi_N')

ens='milc'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def symmetrize(dat):
    dat_bw=np.roll(np.flip(dat,axis=1),1,axis=1)
    return((dat[:,:,0]+dat[:,:,5]) - (dat_bw[:,:,10]+dat_bw[:,:,15]))/4

d_jk=3
Nsrc=100
path=f'/capstor/store/cscs/userlab/lp139/lyan/code/projectData/03_NpiScatteringWilson/MILC_a09m130W/data_merge/NpiScatteringWilson_Nsrc={Nsrc}.h5'
data={}
with h5py.File(path) as f:    
    cfgs_all=[cfg.decode() for cfg in f['cfgs'][:]]
    # cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs]
    
    if True:
        cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs_all]
    
    for cont in f['data'].keys():
        if cont in ['N']:
            t=f[f'data/{cont}'][:,:,16]
            t=symmetrize(t)
            t=np.real(t)
            data[cont]=yu.jackknife(t[cfgs_map],d=d_jk)
            continue
        if cont in ['P']:
            t=f[f'data/{cont}'][:,:,0]
            t_bw=np.roll(np.flip(t,axis=1),1,axis=1)
            t=(t+t_bw)/2
            t=np.real(t)
            data[cont]=yu.jackknife(t[cfgs_map],d=d_jk)
            continue
        t=f[f'data/{cont}'][:,:,0]
        t=symmetrize(t)
        t=np.real(t)
        data[cont]=yu.jackknife(t[cfgs_map],d=d_jk)
        
print(data['N'].shape)

(113, 96)


In [9]:
c2pt=data['P']
meff=yu.jackmap(yu.c2pt2meff_pion,c2pt,yun.ens2NT[ens])
tmins=np.arange(1,35); tmax=40
fits=yu.doFits_2pt_pion(meff,tmins,tmax,corrQ=False)

fig,axs,result=yu.makePlot_2pt_pion(meff,fits,xunit=yun.ens2a[ens],yunit=yun.ens2aInv[ens],selection=round(2/yun.ens2a[ens]))
yu.finalizePlot('meff_pion')
yu.save_pkl_reg('mpi',result)

In [11]:
meff=yu.jackmap(yu.c2pt2meff,data['N'])

tminss=[range(8,22+1),range(1,8+1),range(1,2+1)]
selections={'1st':15,'2st':6,'3st':1}
print(selections)

mN_exp=(yu.m_proton+yu.m_neutron)/2

fitss_2pt=yu.doFits_meff_nst(meff,tminss,[0.4,0.5,2,0.8,1])
fig,axd,result=yu.makePlot_2pt_SimoneStyle(meff,fitss_2pt,xunit=yun.ens2a[ens],yunit=yun.ens2aInv[ens]/1000,mN_exp=mN_exp/1000,ylims='std_N',\
    selection=selections)
fig.suptitle(yun.ens2full[ens])
yu.finalizePlot('meff_N')

pars_jk_meff1st=result['1st']
pars_jk_meff2st=result['2st']
pars_jk_meff3st=result['3st']
    

label=f'pars_jk_meffnst_selected'
yu.save_pkl_reg(label,[pars_jk_meff1st,pars_jk_meff2st,pars_jk_meff3st])

{'1st': 15, '2st': 6, '3st': 1}
